Exploration of the data from the [Di-Tech Challenge](http://research.xiaojukeji.com/competition), organized by Didi Chuxing, a ride-hailing company in China. The data is described [here](http://research.xiaojukeji.com/competition/detail.action?competitionId=DiTech2016).

In [ ]:
import pandas as pd
import numpy as np

# Warn about chained assignment?
# pd.options.mode.chained_assignment = None

from ggplot import *
%matplotlib inline

from time import clock

# Order Info Table

<table>
        <tr>
            <th>Field</th>
            <th>Type</th>
            <th>Meaning</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>order_id</td>
            <td>string</td>
            <td>order ID</td>
            <td>70fc7c2bd2caf386bb50f8fd5dfef0cf</td>
        </tr>
        <tr>
            <td>driver_id</td>
            <td>string</td>
            <td>driver ID</td>
            <td>56018323b921dd2c5444f98fb45509de</td>
        </tr>
        <tr>
            <td>passenger_id</td>
            <td>string</td>
            <td>user ID</td>
            <td>238de35f44bbe8a67bdea86a5b0f4719</td>
        </tr>
        <tr>
            <td>start_district_hash</td>
            <td>string</td>
            <td>departure</td>
            <td>d4ec2125aff74eded207d2d915ef682f</td>
        </tr>
        <tr>
            <td>dest_district_hash</td>
            <td>string</td>
            <td>destination</td>
            <td>929ec6c160e6f52c20a4217c7978f681</td>
        </tr>
        <tr>
            <td>Price</td>
            <td>double</td>
            <td>Price</td>
            <td>37.5</td>
        </tr>
        <tr>
            <td>Time</td>
            <td>string</td>
            <td>Timestamp of the order</td>
            <td>2016-01-15 00:35:11</td>
        </tr>
</table>

The Order Info Table shows the basic information of an order, including the passenger and the driver (if driver_id =NULL, it means the order was not answered by any driver), place of origin, destination, price and time. The fields order_id, driver_id, passenger_id, start_hash, and dest_hash are made not sensitive.

In [ ]:
# Columns in order files
columns = ['order_id', 'driver_id', 'passenger_id', 'start_district_hash', 'dest_district_hash', 'price', 'time']

# Open only one file
# order_file_1 = "data/season_1/training_data/order_data/order_data_2016-01-01"
# df = df_1 = pd.read_csv(order_file_1, sep = "\t", names = columns, parse_dates = 'time')

# Files are organized by dates
n_files = 22
order_files = ["data/season_1/training_data/order_data/order_data_2016-01-{:02d}".format(i) 
               for i in range(1, n_files)]

# Open all of them
order_dfs = []
for order_file in order_files:
    order_dfs.append(pd.read_csv(order_file, sep = "\t", names = columns))
df = pd.concat(order_dfs)

# Recognize time column as time
df['time'] = pd.to_datetime(df.time)
# Set the row labels to the time stamp
# df = df.set_index('time')

# Keep first two weeks for training, next one week for validation.
ind = df['time'] < pd.to_datetime('2016-01-15')
df_train = df[ind]
df_valid = df[~ind]

# Keep a random number of the rows
# df_train = df.sample(frac = 0.70, random_state = 111)
# df_valid = df[~df.index.isin(df_train.index)]

# Avoid looking at validation set during the exploration
df = df_train

# Quick look at the data frame
df.head(2)

In [ ]:
df.get_dtype_counts()

In [ ]:
# How many entries? unique orders/passengers/drivers?

num_entries = df.shape[0]
print("{} entries".format(num_entries))

num_orders = len(df.order_id.unique())
print("{} unique orders ({:.1%})".format(num_orders, num_orders/num_entries))

num_pass = len(df.passenger_id.unique())
print("{} unique passengers ({:.1%})".format(num_pass, num_pass/num_entries))
      
num_drivers = len(df.driver_id.unique())
print("{} unique drivers ({:.1%})".format(num_drivers, num_drivers/num_entries))

num_start_district = len(df.start_district_hash.unique())
print("{} unique starting districts ({:.1%})".format(num_start_district, num_start_district/num_entries))

num_dest_district = len(df.dest_district_hash.unique())
print("{} unique destination districts ({:.1%})".format(num_dest_district, num_dest_district/num_entries))

# Price distribution
print("\n")
print(df.price.describe())

In [ ]:
# Extract the date, and implicitly make the time midnight.
df['date'] = df.time.dt.date
df['timeonly'] = df.time.dt.time

# Compute time slot
# One day is uniformly divided into 144 ten minute time slots. Indexed from 1 to 144.
df['timeslot'] = (df['time'] - pd.to_datetime(df['date'])).astype('timedelta64[m]')//10 + 1

# Drop the time column?
# df = df.drop('time', axis = 1)

# Is it a weekend or weekday?
df['dow'] = df.time.dt.dayofweek
df['weekend'] = df.dow >= 5

df.head(2)

In [ ]:
print("Dates from {} to {}.".format(df['time'].min(), df['time'].max()))

# Gap

In [ ]:
# Count how many rows per order_id and driver_id
count = df[['order_id', 'driver_id']].groupby('order_id').count()
count = count.reset_index()
count = count['driver_id']

# Orders picked up by more than one driver?
print(sum(count > 1))
# Yes..? Surprising.

# Turns out there are duplicate and almost-duplicate entries. 
# The FAQ recommends just leaving them in.

# Remove the duplicates
# dup = df.duplicated(['order_id', 'driver_id', 'passenger_id', 'time'], keep = 'last')
# df = df[~dup]

# Proportion of orders not picked up by a driver
s = sum(count == 0)
l = len(count)

print("There are {} orders-without-drivers out of {} orders: {:.1%}.".format(s, l, s/l))
# The gap is simply the number of orders not picked up.

# FIXME This might be overcounting gap. Some driver_id still get 1?

In [ ]:
# Was order answered?
df['is_gap'] = df['driver_id'].isnull()

# Proportion of orders not picked up by a driver
s = sum(df['is_gap'])
l = len(df['is_gap'])
print("There are {} orders-without-drivers out of {} orders: {:.1%}.".format(s, l, s/l))

# df.head(2)
# df.describe()

In [ ]:
# Plot of gap vs elapsed time in days group by ten minutes interval
df['time_from_begin'] = (df['time'] - pd.to_datetime('2016-01-01')).astype('timedelta64[m]')//10/6/24

cols = ['time_from_begin']
df_select = df[cols + ['is_gap']]
df_gap = df_select.groupby(cols).sum()
df_gap = df_gap.reset_index()

ggplot(aes('time_from_begin', 'is_gap'), data = df_gap) + \
    geom_point(color = 'gray') + \
    geom_line() + \
    xlab('Elapsed time in days') + ylab('Gap')

In [ ]:
# Compute gap per time slot per district
cols = ['start_district_hash', 'date', 'timeslot']

# Number of missing drivers per group 
df['gap'] = df.groupby(cols)['is_gap'].transform('sum')

# df_select = df[cols + ['is_gap']]
# df_gap = df_select.groupby(cols).sum()

# Flatten data frame after the group by
# df_gap = df_gap.reset_index()
# df_gap = df_gap.rename(columns = {'is_gap': 'gap'})

# Sanity check: do the numbers add up?
# print(sum(df_gap.gap))

# Merge back into main data frame
# df = df.merge(df_gap, on = cols, how = 'left')

df.head(2)

# Weather

<table>
        <tr>
            <th>Field</th>
            <th>Type</th>
            <th>Meaning</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>Time</td>
            <td>string</td>
            <td>Timestamp</td>
            <td>2016-01-15 00:35:11</td>
        </tr>
        <tr>
            <td>Weather</td>
            <td>int</td>
            <td>Weather</td>
            <td>7</td>
        </tr>
        <tr>
            <td>temperature</td>
            <td>double</td>
            <td>Temperature</td>
            <td>-9</td>
        </tr>
        <tr>
            <td>PM2.5</td>
            <td>double</td>
            <td>pm25</td>
            <td>66</td>
        </tr>
</table>

The Weather Info Table shows the weather info every 10 minutes each city. The weather field gives the weather conditions such as sunny, rainy, and snowy etc; all sensitive information has been removed. The unit of temperature is Celsius degree, and PM2.5 is the level of air pollutions.

In [ ]:
# Files are organized by dates
n_files = 21
weather_files = ["data/season_1/training_data/weather_data/weather_data_2016-01-{:02d}".format(i)
                 for i in range(1, n_files)]

# Open all of them
columns = ['time', 'weather', 'temperature', 'pm25']
weather_dfs = []
for f in weather_files:
    weather_dfs.append(pd.read_csv(f, sep = "\t", names = columns))
dfw = pd.concat(weather_dfs)

# Extract date and time slot
dfw['time'] = pd.to_datetime(dfw.time)
dfw['date'] = dfw.time.dt.date
dfw['timeslot'] = (dfw['time'] - pd.to_datetime(dfw['date'])).astype('timedelta64[m]')//10 + 1
dfw = dfw.drop('time', axis = 1)

# Merge into main data frame, and fill missing values
# http://pandas.pydata.org/pandas-docs/stable/missing_data.html
df = df.merge(dfw, on = ['date', 'timeslot'], how = 'left')
df.temperature = df.temperature.fillna(method = 'ffill') # forward fill
# df.temperature = df.temperature.interpolate(method = 'time') # time-based interpolation

# Quick peek
dfw.describe()

# Categorical Variables

In [ ]:
# Rank
df['start_district_count'] = df.groupby('start_district_hash')['start_district_hash'].transform('count')
df['start_district_rank'] = df['start_district_count'].rank(ascending = False)
df['district_rank'] = df['start_district_rank']

# Number of most popular districts to keep
k = 10

num_entries = df.shape[0]
df_filtered = df[df['start_district_rank'] < k]
num_top = df_filtered['start_district_count'].sum()

# districts_rank = df.start_district_hash.value_counts(sort = True)

# Look at k top districts
print("The first {} most popular districts account for {} out of {} ({:.1%})".format(
        k, num_top, num_entries, num_top/num_entries))

# Extract most popular districts
districts_top = df_filtered['start_district_hash']

# Look only at most popular districts
# df['district'] = np.nan
# df.loc[df.start_district_hash.isin(districts_top), 'district'] = \
#     df.loc[df.start_district_hash.isin(districts_top), 'start_district_hash']
    
# One-hot encoding
# dummies = pd.get_dummies(df['district'], dummy_na = False)
# df = pd.concat((df.drop('district', axis = 1), dummies.astype(int)), axis = 1)

df.head(2)

# Preparation for Validation

In [ ]:
# Compute time slot
df_valid['time'] = pd.to_datetime(df_valid.time)
df_valid['date'] = df_valid.time.dt.date
df_valid['timeslot'] = (df_valid['time'] - pd.to_datetime(df_valid['date'])).astype('timedelta64[m]')//10 + 1

# Day of week
df_valid['dow'] = df_valid.time.dt.dayofweek
df_valid['weekend'] = df_valid.dow >= 5

# Compute gap per time slot per district
df_valid['is_gap'] = df_valid['driver_id'].isnull()
cols = ['start_district_hash', 'date', 'timeslot']
df_select = df_valid[cols + ['is_gap']]
df_gap = df_select.groupby(cols).sum()
df_gap = df_gap.reset_index().rename(columns = {'is_gap': 'gap'})
df_valid = df_valid.merge(df_gap, on = cols, how = 'left')

# Merge temperature
df_valid = df_valid.merge(dfw, on = ['date', 'timeslot'], how = 'left')
df_valid.temperature = df_valid.temperature.fillna(method = 'ffill')

# One-hot encoding of districts
# df_valid['district'] = np.nan
# df_valid.loc[df_valid.start_district_hash.isin(districts), 'district'] = \
#     df_valid.loc[df_valid.start_district_hash.isin(districts), 'start_district_hash']
# dummies = pd.get_dummies(df_valid['district'], dummy_na = False)
# df_valid = pd.concat((df_valid.drop('district', axis = 1), dummies.astype(int)), axis = 1)

# Replace district by popularity
districts_rank = df[['start_district_hash', 'district_rank']]
districts_rank = districts_rank.drop_duplicates(subset = ['start_district_hash'], keep = 'first')
districts_rank = districts_rank.set_index('start_district_hash')['district_rank']
df_valid['start_district_rank'] = districts_rank[df_valid.start_district_hash].reset_index()['district_rank']

# Predictions by Clusters

In [ ]:
# Make first prediction by simply taking the mean per start_district_hash per timeslot per weekend
cols = ['start_district_hash', 'weekend', 'timeslot']
# cols = ['start_district_hash', 'dow', 'timeslot']

df_select = df[cols + ['gap']]
gap_cluster = df_select.groupby(cols).min().reset_index()
# gap_cluster = df_select.groupby(cols).mean().reset_index()
gap_cluster = gap_cluster.rename(columns = {'gap': 'gap_cluster'})

# Work directly on main data frame
# df['gap_cluster'] = df[cols + ['gap']].groupby(cols)['gap'].transform('min')

# Merge back into main data frame
df_cluster = df.merge(gap_cluster, on = cols, how = 'left')

# Quick look
df.head(2)

# Set given outcome and predictions
train_outcome = df_cluster['gap']
train_predict = df_cluster['gap_cluster']

In [ ]:
# Validation

df_valid_cluster = df_valid[cols + ['gap']].merge(gap_cluster, on = cols, how = 'left')

valid_outcome = df_valid_cluster['gap']
valid_predict = df_valid_cluster['gap_cluster']

# Prediction with sklearn

In [ ]:
# Select features
cols = ['start_district_rank', 'dow', 'timeslot', 'temperature']
# cols = ['start_district_rank', 'weekend', 'timeslot', 'temperature']
train = df[cols]

# Select regressor
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators = 10)
# from sklearn.tree import DecisionTreeRegressor
# reg = DecisionTreeRegressor(max_depth = 3)

# Fit training data
start = clock()
reg.fit(train, train_outcome)
print("Fit in {:.0f} seconds.".format(clock() - start))

# Extrapolate to test data
start = clock()
train_predict = reg.predict(train)
print("Extrapolate in {:.0f} seconds.".format(clock() - start))

# Reference outcome
train_outcome = df['gap']

In [ ]:
# Validation
valid_outcome = df_valid['gap']
valid_predict = reg.predict(df_valid[cols])

# Evaluation
Consider di districts and tj time slots, and the supply-demand gap gapij , and your prediction is sij, we use as the evaluation metrics: 
![MAPE](figures/mape.jpg)
The lowest MAPE will be the best.

In [ ]:
def mape(outcome, predict):
    # Compute MAPE score. Lower is better.
    import numpy as np
    
    # Compute errors summand for summand with nonzero denominator
    diff = (outcome - predict) / outcome.replace({0: np.nan})
    diff = diff.replace({np.nan: 0})
    diff = diff.abs()
    
    # Compute the average over all district and timeslots for which outcome is NONZERO
    nq = len(outcome.nonzero()[0])
    return diff.sum() / nq

# As of June 7th...
# lowest score online is 0.224257,
# 100th is 0.27747,
# 500th is 0.360159.

score = mape(train_outcome, train_predict)
print("Training MAPE: {:.6f}".format(score))

# MAPE on training set
# ['start_district_hash', 'weekend', 'timeslot', 'gap_cluster'] -- 0.003352
# ['start_district_hash', 'dow', 'timeslot', 'gap_cluster'] -- 0.011473
# ['start_district_hash', 'dow', 'timeslot', 'ffill-temperature', DTR] -- 0.777944
# ['start_district_hash', 'dow', 'timeslot', 'ffill-temperature', RFR] -- 0.566997

score = mape(valid_outcome, valid_predict)
print("Validation MAPE: {:.6f}".format(score))

# District Info Table

<table>
        <tr>
            <th>Field</th>
            <th>Type</th>
            <th>Meaning</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>district_hash</td>
            <td>string</td>
            <td>District hash</td>
            <td>90c5a34f06ac86aee0fd70e2adce7d8a</td>
        </tr>
        <tr>
            <td>district_id</td>
            <td>string</td>
            <td>District ID</td>
            <td>1</td>
        </tr>
</table>

The District Info Table shows the information about the districts to be evaluated in the contest. You need to do the prediction given the districts from the District Definition Table. In the submission of the results, you need to map the district hash value to district mapped ID.

In [ ]:
# Use the starting district_hash as the associated disctrict
df['district_hash'] = df['start_district_hash']

# Load district conversion table
district_file = 'data/season_1/training_data/cluster_map/cluster_map'
district = pd.read_csv(district_file, sep = '\t', names = ['district_hash', 'district_id'])

# How many districts?
print(district.shape)

# Replace district_hash by district_id in data frame
df = df.merge(district, on = 'district_hash', how = 'left')
# df = df.drop('district_hash', axis = 1)

df.head(2)

# Output
<table class="table table-2">
        <tr>
            <th>Data name</th>
            <th>Data type</th>
            <th>Example</th>
        </tr>
        <tr>
            <td>District ID</td>
            <td>string</td>
            <td>1,2,3,4 (the same as district mapping ID)</td>
        </tr>
        <tr>
            <td>Time slot</td>
            <td>string</td>
            <td>2016-01-23-1 (The first time slot on Jan. 23rd, 2016; one day is uniformly divided into 144 ten minute time slots)</td>
        </tr>
        <tr>
            <td>Prediction value</td>
            <td>double</td>
            <td>6.0</td>
        </tr>
</table>

In [ ]:
# Make the date - timeslot column
df['datetimeslot'] = df.date.map(str) + '-' + df.timeslot.astype(int).map(str)

# Prepare output file
cols = ['district_id', 'datetimeslot']
final = df[cols + ['gap_cluster']].groupby(cols).mean().reset_index()
final.to_csv("predict.csv", index = False, header = False)